STAT 453: Deep Learning (Spring 2021)  
Instructor: Sebastian Raschka (sraschka@wisc.edu)  

Course website: http://pages.stat.wisc.edu/~sraschka/teaching/stat453-ss2021/  
GitHub repository: https://github.com/rasbt/stat453-deep-learning-ss21

---

# Convolutional Autoencoder

## Imports

In [2]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy


#### Import utility functions

In [3]:
from helper_data_loader import get_dataloaders_mnist, read_sr_data, transform_data
from helper_utils import set_deterministic, set_all_seeds
from helper_train import train_autoencoder_v1, get_est, get_data_complete, train_vae_v1, train_v1
from helper_plotting import plot_latent_space_with_labels, plot_frequencies, plot_loss, plot_freq_loss, plot_season, plot_training_loss, plot_cases, plot_residual_loss, plot_noise
from helper_metrics import corr_noise
from helper_log_model import log_result
from helper_models import VQVAE, VAE, AutoEncoder

#### Weight and bias

In [4]:
root = ""#'D:\GIT\pytorch\dl_python'
component = "EW"
data_path = 'data'
fit_sr = '\\SR_table_' + component +'_DL.mat'
filepath = root+ data_path + fit_sr;


In [5]:
##########################
### SETTINGS
##########################

# Device
DEVICE_CPU = "cpu"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', DEVICE)

# Hyperparameters
RANDOM_SEED = 123
LEARNING_RATE = 0.001
BATCH_SIZE = 32 * 20
NUM_EPOCHS = 100
num_hiddens = 64
latent_dim = 10
kernel_size = 32
regression_hidden = 128
freq_mult = 100
dropout = 0.0
SWEEP = 0
SAVE_DATA = 1

Device: cuda


In [6]:
import wandb

config = {
    "learning_rate": LEARNING_RATE,
    "epochs": NUM_EPOCHS,
    "batch_size": BATCH_SIZE,
    "kernel_size": kernel_size,
    "num_hiddens": num_hiddens,
    "lantent_dim": latent_dim,
    "regression_hidden":regression_hidden,
    "freq_mult": freq_mult,
     "dropout": dropout,
    "best_perc": 20,
    "type": "NO_RAW"
}


In [7]:
set_deterministic
set_all_seeds(RANDOM_SEED)

## Dataset

In [8]:
##########################
### Dataset
##########################

train_loader, test_loader, fixed_normal_data, normal_loader = get_dataloaders_mnist(filepath ,batch_size = BATCH_SIZE, case = "NORMAL")

(79281,)
bool
(40055, 512)


In [9]:
train_loader_worst, test_loader_worst, fixed_worst_data, worst_loader = get_dataloaders_mnist(filepath, batch_size = BATCH_SIZE,  case = "WORST")

(79281,)
bool
(10577, 512)


In [10]:
_, _, _, all_loader = get_dataloaders_mnist(filepath, batch_size = BATCH_SIZE,  case = "ALL")

(79281,)
bool
(79281, 512)


In [11]:
# Checking the dataset
print('Training Set:\n')
for data_raw, data_lorentz,values_freq, labels in train_loader:  
    print('data raw dimensions:', data_raw.size())
    batch_size = data_raw.size()[0]
    print('data lorentz dimensions:', data_lorentz.size())
    print('Freq  dimensions:', values_freq.size())
    print('label dimensions:', labels.size())
    NUM_CLASSES = len(numpy.unique(labels))
    print(data_raw[:10])
    break
print("Train len: "  + str(len(train_loader) * batch_size))
print("Classes: "  + str(NUM_CLASSES))
# Checking the dataset
print('\nTesting Set:')
for data_raw, data_lorentz,values_freq,  labels in test_loader:  
    print('data raw dimensions:', data_raw.size())
    print('data lorentz dimensions:', data_lorentz.size())
    print('Freq  dimensions:', values_freq.size())
    print('label dimensions:', labels.size())
    break

Training Set:

data raw dimensions: torch.Size([640, 1, 256])
data lorentz dimensions: torch.Size([640, 1, 256])
Freq  dimensions: torch.Size([640, 6])
label dimensions: torch.Size([640])
tensor([[[0.9050, 1.0000, 0.8184,  ..., 0.1939, 0.1213, 0.0413]],

        [[0.7723, 0.6929, 0.7319,  ..., 0.2167, 0.1728, 0.0889]],

        [[0.8146, 0.8118, 0.8859,  ..., 0.0976, 0.0694, 0.0000]],

        ...,

        [[0.7722, 0.9648, 0.8685,  ..., 0.1396, 0.0948, 0.0374]],

        [[0.9261, 0.8936, 0.8567,  ..., 0.2271, 0.1336, 0.1004]],

        [[1.0000, 0.9721, 0.8276,  ..., 0.1427, 0.0981, 0.0235]]])
Train len: 26240
Classes: 4

Testing Set:
data raw dimensions: torch.Size([640, 1, 256])
data lorentz dimensions: torch.Size([640, 1, 256])
Freq  dimensions: torch.Size([640, 6])
label dimensions: torch.Size([640])


## Model

## VQ-VAE

In [12]:
class VectorQuantizer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, commitment_cost):
        super(VectorQuantizer, self).__init__()
        
        self._embedding_dim = embedding_dim
        self._num_embeddings = num_embeddings
        
        self._embedding = nn.Embedding(self._num_embeddings, self._embedding_dim)
        self._embedding.weight.data.uniform_(-1/self._num_embeddings, 1/self._num_embeddings)
        self._commitment_cost = commitment_cost

    def forward(self, inputs):
        # convert inputs from BCL -> BLC
        inputs = inputs.permute(0, 2, 1).contiguous()
        input_shape = inputs.shape
        #print(input_shape)
        
        # Flatten input
        flat_input = inputs.view(-1, self._embedding_dim)
        
        # Calculate distances
        distances = (torch.sum(flat_input**2, dim=1, keepdim=True) 
                    + torch.sum(self._embedding.weight**2, dim=1)
                    - 2 * torch.matmul(flat_input, self._embedding.weight.t()))
            
        # Encoding
        encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
        encodings = torch.zeros(encoding_indices.shape[0], self._num_embeddings, device=inputs.device)
        encodings.scatter_(1, encoding_indices, 1)
        
        # Quantize and unflatten
        quantized = torch.matmul(encodings, self._embedding.weight).view(input_shape)
        
        # Loss
        e_latent_loss = F.mse_loss(quantized.detach(), inputs)
        q_latent_loss = F.mse_loss(quantized, inputs.detach())
        loss = q_latent_loss + self._commitment_cost * e_latent_loss
        
        quantized = inputs + (quantized - inputs).detach()
        avg_probs = torch.mean(encodings, dim=0)
        perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))
        
        # convert quantized from BLC -> BCL
        return loss, quantized.permute(0, 2, 1).contiguous(), perplexity, encodings
class VQVAE(nn.Module):
    def __init__(self, num_hiddens, kernel_size, data_len, latent_dim, regression_hidden, num_embeddings, dropout = 0, commitment_cost = 0.25):
        super().__init__()
        self.encoder = nn.Sequential( #784
            nn.Conv1d(in_channels=1,
                  out_channels=num_hiddens // 2,
                  kernel_size=kernel_size, stride=2, padding = kernel_size  // 2 - 1, bias=False),
            nn.LeakyReLU(0.01),
            nn.Dropout(dropout),
            nn.Conv1d(in_channels=num_hiddens // 2,
                  out_channels=num_hiddens,
                  kernel_size=kernel_size // 2, stride=2, padding = kernel_size  // 2 // 2 - 1, bias=False),
            nn.LeakyReLU(0.01),
            nn.Conv1d(in_channels=num_hiddens,
                  out_channels=num_hiddens,
                  kernel_size=kernel_size // 4, stride=2, padding = kernel_size  // 4 // 2 - 1, bias=False),
            nn.LeakyReLU(0.01),
        )
        self.pre_vq_conv = nn.Conv1d(in_channels=num_hiddens, 
                                  out_channels=latent_dim,
                                  kernel_size=1, 
                                  stride=1)
        
        self.vq_vae = VectorQuantizer(num_embeddings, latent_dim,
                                       commitment_cost)
        self.regresion = nn.Sequential(
                nn.Flatten(),
                nn.Linear(latent_dim * (data_len // (2 ** 3)), regression_hidden),
                nn.Linear(regression_hidden , 6),
                nn.Sigmoid()
                )
        self.decoder = nn.Sequential(
            nn.Conv1d(in_channels=latent_dim,
                         out_channels=num_hiddens,
                         kernel_size=3, 
                         stride=1, padding=1),
            #torch.nn.Linear(num_hiddens, num_hiddens * (data_len // (2 ** 3))),
            Reshape(-1, num_hiddens, (data_len // (2 ** 3))),
            nn.ConvTranspose1d(in_channels=num_hiddens,
                  out_channels=num_hiddens,
                  kernel_size=kernel_size // 4, stride=2, padding = kernel_size // 4 // 2 - 1, bias=False),
            nn.LeakyReLU(0.01),
            nn.ConvTranspose1d(in_channels=num_hiddens,
                  out_channels=num_hiddens // 2,
                  kernel_size=kernel_size // 2, stride=2, padding = kernel_size // 2 // 2 - 1, bias=False),
            nn.LeakyReLU(0.01),
            nn.ConvTranspose1d(in_channels=num_hiddens // 2,
                  out_channels=1,
                  kernel_size=kernel_size , stride=2, padding = kernel_size  // 2 - 1, bias=False),
            nn.LeakyReLU(0.01),
            nn.Dropout(dropout),
            Trim(),  # 1x29x29 -> 1x28x28
            nn.Sigmoid()
            )

    def forward(self, x):
        x = self.encoder(x)
        z = self.pre_vq_conv(x)
        loss, encoded, perplexity, encoding = self.vq_vae(z)
        #print(encoded.shape)
        freq = self.regresion(encoded)
        decoded = self.decoder(encoded)
        return encoded,perplexity,loss,freq, decoded

In [13]:
## AE

In [14]:
##########################
### MODEL
##########################
class AutoEncoder(nn.Module):
    def __init__(self, num_hiddens, kernel_size, data_len, latent_dim, regression_hidden, dropout):
        super().__init__()
        
        self.encoder = nn.Sequential( #784
                nn.Conv1d(in_channels=1,
                      out_channels=num_hiddens // 2,
                      kernel_size=kernel_size, stride=2, padding = kernel_size  // 2 - 1, bias=False),
                nn.BatchNorm1d(num_hiddens // 2),
                nn.LeakyReLU(0.01),
                nn.Dropout(dropout),
                nn.Conv1d(in_channels=num_hiddens // 2,
                      out_channels=num_hiddens,
                      kernel_size=kernel_size // 2, stride=2, padding = kernel_size  // 2 // 2 - 1, bias=False),
                nn.LeakyReLU(0.01),
                nn.Conv1d(in_channels=num_hiddens,
                      out_channels=num_hiddens,
                      kernel_size=kernel_size // 4, stride=2, padding = kernel_size  // 4 // 2 - 1, bias=False),
                nn.BatchNorm1d(num_hiddens),
                nn.LeakyReLU(0.01),
                nn.Flatten(),
                nn.Linear(num_hiddens * (data_len // (2 ** 3)), latent_dim),
        )
        self.regresion = nn.Sequential(
                nn.Linear(latent_dim , regression_hidden),
                nn.Linear(regression_hidden , 6),
                nn.Sigmoid()
        )
        self.decoder = nn.Sequential(
                torch.nn.Linear(latent_dim, num_hiddens * (data_len // (2 ** 3))),
                Reshape(-1, num_hiddens, (data_len // (2 ** 3))),
                nn.ConvTranspose1d(in_channels=num_hiddens,
                      out_channels=num_hiddens,
                      kernel_size=kernel_size // 4, stride=2, padding = kernel_size // 4 // 2 - 1, bias=False),
                nn.BatchNorm1d(num_hiddens),
                nn.LeakyReLU(0.01),
                nn.ConvTranspose1d(in_channels=num_hiddens,
                      out_channels=num_hiddens // 2,
                      kernel_size=kernel_size // 2, stride=2, padding = kernel_size // 2 // 2 - 1, bias=False),
                nn.BatchNorm1d(num_hiddens // 2),
                nn.LeakyReLU(0.01),
                nn.ConvTranspose1d(in_channels=num_hiddens // 2,
                      out_channels=1,
                      kernel_size=kernel_size , stride=2, padding = kernel_size  // 2 - 1, bias=False),
                nn.LeakyReLU(0.01),
                nn.Dropout(dropout),
                Trim(),  # 1x29x29 -> 1x28x28
                nn.Sigmoid()
                )
    def forward(self, x):
        encoder = self.encoder(x)
        freq = self.regresion(encoder)
        decoder = self.decoder(encoder)
        return encoder,None, None, freq, decoder
    
    

class Reshape(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.shape = args

    def forward(self, x):
        return x.view(self.shape)


class Trim(nn.Module):
    def __init__(self, *args):
        super().__init__()

    def forward(self, x):
        return x[:, :, :256]

In [15]:
## VAE

In [16]:
class VAE(nn.Module):
    def __init__(self, num_hiddens, kernel_size, data_len, latent_dim, regression_hidden, dropout):
        super().__init__()
        
        self.encoder = nn.Sequential( #784
                nn.Conv1d(in_channels=1,
                      out_channels=num_hiddens // 2,
                      kernel_size=kernel_size, stride=2, padding = kernel_size  // 2 - 1, bias=False),
                nn.LeakyReLU(0.01),
                nn.BatchNorm1d(num_hiddens // 2),
                nn.Dropout(dropout),
                nn.Conv1d(in_channels=num_hiddens // 2,
                      out_channels=num_hiddens,
                      kernel_size=kernel_size // 2, stride=2, padding = kernel_size  // 2 // 2 - 1, bias=False),
                nn.LeakyReLU(0.01),
                nn.Conv1d(in_channels=num_hiddens,
                      out_channels=num_hiddens,
                      kernel_size=kernel_size // 4, stride=2, padding = kernel_size  // 4 // 2 - 1, bias=False),
                nn.BatchNorm1d(num_hiddens),
                nn.LeakyReLU(0.01),
                nn.Flatten(),
        )
        self.z_mean = torch.nn.Linear(num_hiddens * (data_len // (2 ** 3)), latent_dim)
        self.z_log_var = torch.nn.Linear(num_hiddens * (data_len // (2 ** 3)), latent_dim)
        
        self.regresion = nn.Sequential(
                nn.Linear(latent_dim , regression_hidden),
                nn.Linear(regression_hidden , 6),
                nn.Sigmoid()
        )
        self.decoder = nn.Sequential(
                torch.nn.Linear(latent_dim, num_hiddens * (data_len // (2 ** 3))),
                Reshape(-1, num_hiddens, (data_len // (2 ** 3))),
                nn.ConvTranspose1d(in_channels=num_hiddens,
                      out_channels=num_hiddens,
                      kernel_size=kernel_size // 4, stride=2, padding = kernel_size // 4 // 2 - 1, bias=False),
                nn.BatchNorm1d(num_hiddens),
                nn.LeakyReLU(0.01),
                nn.ConvTranspose1d(in_channels=num_hiddens,
                      out_channels=num_hiddens // 2,
                      kernel_size=kernel_size // 2, stride=2, padding = kernel_size // 2 // 2 - 1, bias=False),
                nn.BatchNorm1d(num_hiddens // 2),
                nn.LeakyReLU(0.01),
                nn.ConvTranspose1d(in_channels=num_hiddens // 2,
                      out_channels=1,
                      kernel_size=kernel_size , stride=2, padding = kernel_size  // 2 - 1, bias=False),
                nn.LeakyReLU(0.01),
                nn.Dropout(dropout),
                Trim(),  # 1x29x29 -> 1x28x28
                nn.Sigmoid()
                )
    def encoding_fn(self, x):
        x = self.encoder(x)
        z_mean, z_log_var = self.z_mean(x), self.z_log_var(x)
        encoded = self.reparameterize(z_mean, z_log_var)
        return encoded
    def reparameterize(self, z_mu, z_log_var):
        eps = torch.randn(z_mu.size(0), z_mu.size(1), device=z_mu.device)
        z = z_mu + eps * torch.exp(z_log_var/2.) 
        return z
    def latent_sample(self, z_mu, z_log_var):
        # the reparameterization trick
        std = z_log_var.mul(0.5).exp_()
        eps = torch.empty_like(std).normal_()
        return eps.mul(std).add_(z_mu)

    def forward(self, x):
        x = self.encoder(x)
        z_mean, z_log_var = self.z_mean(x), self.z_log_var(x)
        encoded = self.latent_sample(z_mean, z_log_var)
        freq = self.regresion(encoded)#torch.cat((z_mean, z_log_var), dim = 1))
        decoded = self.decoder(encoded)
        return encoded,z_mean,z_log_var,freq, decoded

    
class AutoEncoder(nn.Module):
    def __init__(self, num_hiddens, kernel_size, data_len, latent_dim, regression_hidden, dropout):
        super().__init__()
        
        self.encoder = nn.Sequential( #784
                nn.Conv1d(in_channels=1,
                      out_channels=num_hiddens // 2,
                      kernel_size=kernel_size, stride=2, padding = kernel_size  // 2 - 1, bias=False),
                nn.BatchNorm1d(num_hiddens // 2),
                nn.LeakyReLU(0.01),
                nn.Dropout(dropout),
                nn.Conv1d(in_channels=num_hiddens // 2,
                      out_channels=num_hiddens,
                      kernel_size=kernel_size // 2, stride=2, padding = kernel_size  // 2 // 2 - 1, bias=False),
                nn.LeakyReLU(0.01),
                nn.Conv1d(in_channels=num_hiddens,
                      out_channels=num_hiddens,
                      kernel_size=kernel_size // 4, stride=2, padding = kernel_size  // 4 // 2 - 1, bias=False),
                nn.BatchNorm1d(num_hiddens),
                nn.LeakyReLU(0.01),
                nn.Flatten(),
                nn.Linear(num_hiddens * (data_len // (2 ** 3)), latent_dim),
        )
        self.regresion = nn.Sequential(
                nn.Linear(latent_dim , regression_hidden),
                nn.Linear(regression_hidden , 6),
                nn.Sigmoid()
        )
        self.decoder = nn.Sequential(
                torch.nn.Linear(latent_dim, num_hiddens * (data_len // (2 ** 3))),
                Reshape(-1, num_hiddens, (data_len // (2 ** 3))),
                nn.ConvTranspose1d(in_channels=num_hiddens,
                      out_channels=num_hiddens,
                      kernel_size=kernel_size // 4, stride=2, padding = kernel_size // 4 // 2 - 1, bias=False),
                nn.BatchNorm1d(num_hiddens),
                nn.LeakyReLU(0.01),
                nn.ConvTranspose1d(in_channels=num_hiddens,
                      out_channels=num_hiddens // 2,
                      kernel_size=kernel_size // 2, stride=2, padding = kernel_size // 2 // 2 - 1, bias=False),
                nn.BatchNorm1d(num_hiddens // 2),
                nn.LeakyReLU(0.01),
                nn.ConvTranspose1d(in_channels=num_hiddens // 2,
                      out_channels=1,
                      kernel_size=kernel_size , stride=2, padding = kernel_size  // 2 - 1, bias=False),
                nn.LeakyReLU(0.01),
                nn.Dropout(dropout),
                Trim(),  # 1x29x29 -> 1x28x28
                nn.Sigmoid()
                )
    def forward(self, x):
        encoder = self.encoder(x)
        freq = self.regresion(encoder)
        decoder = self.decoder(encoder)
        return encoder,None, None, freq, decoder

In [17]:
from torchvision import models
from torchsummary import summary
data_len = 256
encoded_type = "RAW"
model_type = "AE"
#optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  
model = AutoEncoder(num_hiddens = num_hiddens, kernel_size = kernel_size, data_len = data_len, latent_dim = latent_dim, 
                         regression_hidden = regression_hidden, dropout = dropout)

summary(model, (1,256), batch_size=-1, device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 32, 128]           1,024
       BatchNorm1d-2              [-1, 32, 128]              64
         LeakyReLU-3              [-1, 32, 128]               0
           Dropout-4              [-1, 32, 128]               0
            Conv1d-5               [-1, 64, 64]          32,768
         LeakyReLU-6               [-1, 64, 64]               0
            Conv1d-7               [-1, 64, 32]          32,768
       BatchNorm1d-8               [-1, 64, 32]             128
         LeakyReLU-9               [-1, 64, 32]               0
          Flatten-10                 [-1, 2048]               0
           Linear-11                   [-1, 10]          20,490
           Linear-12                  [-1, 128]           1,408
           Linear-13                    [-1, 6]             774
          Sigmoid-14                   

In [18]:
data_len = 256
encoded_type = "RAW"
model_type = "VAE"
SWEEP = 0
wandb.init(project="VAE", notes="EW",tags=["VAE", "paper7"],config=config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ual-ucv-uma (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [19]:
NUM_EPOCHS = 2000
SAVE_DATA = 1


## Training

In [20]:
if SWEEP == 1:
    sweep_config = {
      "name" : "regression_lantent",
      "method" : "grid",
      "parameters" : {
          "num_embeddings":{
              "values" : [32]
          },
          "commitment_cost":{
              "values" : [ 0.7]
          },
        "encoded_type" : {
          "values" : ['RAW', 'LORENTZ']
        },
         "model_type" : {
          "values" : ["AE","VAE","VQVAE"]
        }
      }
    }

    sweep_id = wandb.sweep(sweep_config)
    def train():
        config = {
            "learning_rate": LEARNING_RATE,
            "epochs": NUM_EPOCHS,
            "batch_size": BATCH_SIZE,
            "kernel_size": kernel_size,
            "num_hiddens": num_hiddens,
            "lantent_dim": latent_dim,
            "regression_hidden":regression_hidden,
            "freq_mult": freq_mult,
             "dropout": dropout,
            "best_perc": 20,
        }
        with wandb.init(project="Six Cases", notes="FinishingResults",tags=["VQVAEAE", "paper5"],config=config) as run:
            config = wandb.config
            model_type =  config["model_type"]
            encoded_type = config["encoded_type"]
            if model_type == "AE":
                model = AE(num_hiddens = num_hiddens, kernel_size = kernel_size, data_len = data_len, latent_dim = latent_dim, 
                                 regression_hidden = regression_hidden, dropout = dropout)
            if model_type == "VAE":
                model = VAE(num_hiddens = num_hiddens, kernel_size = kernel_size, data_len = data_len, latent_dim = latent_dim,
                            regression_hidden = regression_hidden, dropout = dropout)
            if model_type == "VQVAE":
                model = VQVAE(num_hiddens = num_hiddens, kernel_size = kernel_size, data_len = data_len, latent_dim = latent_dim,
                      regression_hidden = regression_hidden, num_embeddings = config["num_embeddings"], dropout = 0, commitment_cost = config["commitment_cost"])
            model.to(DEVICE)
            optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) 
            log_dict = train_v1(num_epochs=NUM_EPOCHS, model=model, 
                                    optimizer=optimizer, device=DEVICE, freq_mult = freq_mult,
                                    train_loader=train_loader,
                                    worst_case_loader = train_loader_worst,
                                    skip_epoch_stats=True,
                                    logging_interval=200, wandb = wandb, model_type = model_type, encoded_type = encoded_type)
            if SAVE_DATA == 1:
                data = get_data_complete(filepath, model, "cpu", )
                torch.save(model, "models/"+ run.name + ".model")
                log_result(wandb,NUM_EPOCHS,  model, DEVICE,data, log_dict,train_loader, test_loader, train_loader_worst,  model_type, encoded_type)

    
    wandb.agent(sweep_id, function=train)
if SWEEP == 0:
    from torchvision import models
    from torchsummary import summary
    set_all_seeds(RANDOM_SEED)

    if model_type == "AE":
        model = AE(num_hiddens = num_hiddens, kernel_size = kernel_size, data_len = data_len, latent_dim = latent_dim, 
                         regression_hidden = regression_hidden, dropout = dropout)
    if model_type == "VAE":
        model = VAE(num_hiddens = num_hiddens, kernel_size = kernel_size, data_len = data_len, latent_dim = latent_dim,
                    regression_hidden = regression_hidden, dropout = dropout)
    if model_type == "VQVAE":
        model = VQVAE(num_hiddens = num_hiddens, kernel_size = kernel_size, data_len = data_len, latent_dim = latent_dim,
                      regression_hidden = regression_hidden, num_embeddings = 32, dropout = 0, commitment_cost = 0.25)
    model.to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  
    #summary(model, (1,256), batch_size=-1, device='cuda')


    log_dict = train_v1(num_epochs=NUM_EPOCHS, model=model, 
                            optimizer=optimizer, device=DEVICE, freq_mult = freq_mult,
                            train_loader=train_loader,
                            worst_case_loader = train_loader_worst,
                            skip_epoch_stats=True,
                            logging_interval=200, wandb = wandb,model_type = model_type, encoded_type = encoded_type)
    if SAVE_DATA == 1:
        data = get_data_complete(filepath, all_loader,  model, DEVICE)
        torch.save(model, "models/"+ wandb.run.name + "_" + component +  ".model")
        log_result(wandb,NUM_EPOCHS,  model, DEVICE,data, log_dict,train_loader, test_loader, train_loader_worst,  model_type, encoded_type)

Time elapsed: 0.14 min
Epocah: 001/2000 | Loss: 0.3429 | Loss_freq: 0.2403 | Loss_ae: 0.0555 | kl_div 0.0471 | Worst_Loss: 0.0555 | Worst_Loss_freq: 0.0776 | Worst_Loss_ae: 0.1331 |
Time elapsed: 15.16 min
Epocah: 201/2000 | Loss: 0.0457 | Loss_freq: 0.0295 | Loss_ae: 0.0023 | kl_div 0.0138 | Worst_Loss: 0.0247 | Worst_Loss_freq: 0.0082 | Worst_Loss_ae: 0.0330 |
Time elapsed: 30.20 min
Epocah: 401/2000 | Loss: 0.1214 | Loss_freq: 0.0917 | Loss_ae: 0.0032 | kl_div 0.0265 | Worst_Loss: 0.0320 | Worst_Loss_freq: 0.0094 | Worst_Loss_ae: 0.0415 |
Time elapsed: 45.25 min
Epocah: 601/2000 | Loss: 0.0388 | Loss_freq: 0.0270 | Loss_ae: 0.0021 | kl_div 0.0097 | Worst_Loss: 0.0229 | Worst_Loss_freq: 0.0077 | Worst_Loss_ae: 0.0306 |
Time elapsed: 60.31 min
Epocah: 801/2000 | Loss: 0.0320 | Loss_freq: 0.0194 | Loss_ae: 0.0022 | kl_div 0.0104 | Worst_Loss: 0.0274 | Worst_Loss_freq: 0.0077 | Worst_Loss_ae: 0.0351 |
Time elapsed: 75.36 min
Epocah: 1001/2000 | Loss: 0.0253 | Loss_freq: 0.0128 | Loss_ae

TypeError: get_data_complete() missing 1 required positional argument: 'device'

In [22]:
torch.save(model, "models/"+ wandb.run.name + "_" + component +  ".model")

In [ ]:
log_result(wandb,NUM_EPOCHS,  model, DEVICE,data, log_dict,train_loader, test_loader, train_loader_worst,  model_type, encoded_type)

In [ ]:
model = VQVAE(num_hiddens = num_hiddens, kernel_size = kernel_size, data_len = data_len, latent_dim = latent_dim,
              regression_hidden = regression_hidden, embedding_dim = 64, num_embeddings = 32, dropout = 0)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  
summary(model, (1,256), batch_size=-1, device='cuda')

In [ ]:
metric = corr_noise(data)
print(metric)

In [ ]:
metric


## Evaluation

In [ ]:
model_type = "VAE"
encoded_type = "RAW"
base_path = "images/" + model_type + "_" + encoded_type + "_";
path_loss_batch = base_path + "loss.png"
plot_loss(log_dict, NUM_EPOCHS, model_type).savefig(path_loss_batch)
wandb.log({"loss_batch": wandb.Image(path_loss_batch)})

path_worst_residual_freq_loss = base_path + "worst_residual_freq_loss.png"
plot_residual_loss(test_loader_worst, model, DEVICE, model_type = model_type, show = False).savefig(path_worst_residual_freq_loss)
wandb.log({"worst_residual_freq_loss": wandb.Image(path_worst_residual_freq_loss)})


path_test_residual_freq_loss = base_path + "test_residual_freq_loss.png"
plot_residual_loss(test_loader, model, DEVICE, model_type = model_type, show = False).savefig(path_test_residual_freq_loss)
wandb.log({"test_residual_freq_loss": wandb.Image(path_test_residual_freq_loss)})

path_train_freq_qqplot = base_path + "train_freq_qqplot.png"
plot_freq_loss(train_loader, model, DEVICE, model_type = model_type, show = False).savefig(path_train_freq_qqplot)
wandb.log({"train_freq_qqplot": wandb.Image(path_train_freq_qqplot)})

path_worst_freq_qqplot = base_path + "worst_freq_qqplot.png"
plot_freq_loss(train_loader_worst, model, DEVICE, model_type = model_type, show = False).savefig(path_worst_freq_qqplot)
wandb.log({"worst_freq_qqplot": wandb.Image(path_worst_freq_qqplot)})

path_worst_cases = base_path + "worst_cases.png"
plot_cases(test_loader_worst, model, DEVICE, model_type = model_type).savefig(path_worst_cases)
wandb.log({"worst_cases": wandb.Image(path_worst_cases)})

path_test_cases = base_path + "test_cases.png"
plot_cases(test_loader, model, DEVICE, model_type = model_type).savefig(path_test_cases)
wandb.log({"test_cases": wandb.Image(path_test_cases)})

dict_freq = selected_freq(data)

path_hist_freq = base_path + "hist_freq.png"
plot_hist_freq(data).savefig(path_hist_freq)
wandb.log({"hist_freq": wandb.Image(path_hist_freq)})

for index_sr_mode in range(1,7):
    freq_qqplot = base_path + str(index_sr_mode) + "_freq_qqplot.png"
    plot_season(data, sr_mode=index_sr_mode, show = False).savefig(freq_qqplot)
    wandb.log({ str(index_sr_mode) + "_freq_qqplot": wandb.Image(freq_qqplot)})

In [ ]:
log_result(wandb,NUM_EPOCHS,  model, DEVICE,data, log_dict,train_loader, test_loader, train_loader_worst,  model_type, encoded_type)

In [ ]:
from helper_log_model import log_result
from sklearn import preprocessing    
import numpy as np

In [ ]:
data_test = read_sr_data(filepath)

In [28]:
model = torch.load("models/AE_RAW.model")
data = get_data_complete(filepath, model, "cpu")
List = [model, data];
Dict = {"AE_RAW":List}

(79281, 512)


In [56]:
from helper_models import VQVAE, VAE, AutoEncoder, Reshape, Trim, VectorQuantizer
model = torch.load("models/VQVAE_RAW.model")
values =  get_est(all_loader, model, DEVICE)

In [ ]:
from helper_models import VQVAE, VAE, AutoEncoder, Reshape, Trim, VectorQuantizer
from helper_train import get_data_complete
import torch
# load pickle module
import pickle

root = ""
data_path = 'data'
fit_sr = '\\SR_table_DL.mat'
datapath = filepath = root + data_path + fit_sr;
start_model_path = "models/"
end_model_path = ".model"
models = ["AE_RAW", "VAE_RAW", "VQVAE_RAW", "AE_LORENTZ", "VAE_LORENTZ", "VQVAE_LORENTZ"]
models_dict = {}
for index_model in range(6):
    model_name = models[index_model] 
    model_path = start_model_path + model_name + end_model_path
    print(model_path)
    model = torch.load(model_path)
    data = get_data_complete(filepath, all_loader, model, "cpu")
    current_dict = {"model": model, "data": data}
    models_dict[model_name] = current_dict
        
# create a binary pickle file 
f = open("models.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(models_dict,f)

# close file
f.close()


models/AE_RAW.model
models/VAE_RAW.model
